In [1]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd

df = pd.read_csv("s3://science-hackathon-2022-duc-nguyen/train_2v.csv")

In [3]:
df['stroke'].value_counts() 

0    42617
1      783
Name: stroke, dtype: int64

In [4]:
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
43395,56196,Female,10.0,0,0,No,children,Urban,58.64,20.4,never smoked,0
43396,5450,Female,56.0,0,0,Yes,Govt_job,Urban,213.61,55.4,formerly smoked,0
43397,28375,Female,82.0,1,0,Yes,Private,Urban,91.94,28.9,formerly smoked,0
43398,27973,Male,40.0,0,0,Yes,Private,Urban,99.16,33.2,never smoked,0


In [5]:
df['work_type'].value_counts()

Private          24834
Self-employed     6793
children          6156
Govt_job          5440
Never_worked       177
Name: work_type, dtype: int64

In [1]:
import h2o
from h2o.automl import H2OAutoML


# Start the H2O cluster (locally)
h2o.init()

# Import a sample binary outcome train/test set into H2O
data = h2o.import_file("s3://science-hackathon-2022-duc-nguyen/train_2v.csv")
train, test = data.split_frame(ratios=[0.8])

# Identify predictors and response
x = train.columns
y = "stroke"
x.remove(y)
x.remove("id")

# For binary classification, response should be a factor
train[y] = train[y].asfactor()

# Run AutoML for 20 base models
aml = H2OAutoML(max_models=300,max_runtime_secs=1000, seed=2022,balance_classes=True, max_after_balance_size=100, sort_metric = "aucpr")
aml.train(x=x, y=y, training_frame=train)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows) 


Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,7 mins 57 secs
H2O_cluster_timezone:,UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.2
H2O_cluster_version_age:,30 days
H2O_cluster_name:,H2O_from_python_ec2_user_jn0mhk
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,771 Mb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


model_id,aucpr,auc,logloss,mean_per_class_error,rmse,mse
GBM_5_AutoML_2_20220625_173102,0.0924703,0.845686,0.0762736,0.356395,0.130528,0.0170375
GBM_2_AutoML_2_20220625_173102,0.090107,0.837554,0.0785719,0.393705,0.130937,0.0171445
GBM_1_AutoML_2_20220625_173102,0.0895469,0.83718,0.0822016,0.382694,0.1315,0.0172923
GLM_1_AutoML_2_20220625_173102,0.0836616,0.850757,0.0744423,0.333709,0.130339,0.0169883
XRT_1_AutoML_2_20220625_173102,0.0785876,0.835737,0.0921992,0.401537,0.132979,0.0176835
GBM_3_AutoML_2_20220625_173102,0.0761963,0.825213,0.0811521,0.390585,0.131452,0.0172796
GBM_4_AutoML_2_20220625_173102,0.0718198,0.817807,0.0858251,0.350339,0.132112,0.0174537
DeepLearning_1_AutoML_2_20220625_173102,0.0661231,0.825966,0.0783862,0.331149,0.131468,0.0172839
XGBoost_1_AutoML_2_20220625_173102,0.0658629,0.795778,0.116098,0.417953,0.168336,0.0283371
XGBoost_3_AutoML_2_20220625_173102,0.0635095,0.781511,0.126929,0.432541,0.171993,0.0295815


In [2]:
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
lb

model_id,aucpr,auc,logloss,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
GBM_5_AutoML_2_20220625_173102,0.0924703,0.845686,0.0762736,0.356395,0.130528,0.0170375,1537,0.017361,GBM
GBM_2_AutoML_2_20220625_173102,0.090107,0.837554,0.0785719,0.393705,0.130937,0.0171445,2561,0.01802,GBM
GBM_1_AutoML_2_20220625_173102,0.0895469,0.83718,0.0822016,0.382694,0.1315,0.0172923,12853,0.016416,GBM
GLM_1_AutoML_2_20220625_173102,0.0836616,0.850757,0.0744423,0.333709,0.130339,0.0169883,4600,0.001373,GLM
XRT_1_AutoML_2_20220625_173102,0.0785876,0.835737,0.0921992,0.401537,0.132979,0.0176835,4461,0.023107,DRF
GBM_3_AutoML_2_20220625_173102,0.0761963,0.825213,0.0811521,0.390585,0.131452,0.0172796,1510,0.015026,GBM
GBM_4_AutoML_2_20220625_173102,0.0718198,0.817807,0.0858251,0.350339,0.132112,0.0174537,1678,0.013916,GBM
DeepLearning_1_AutoML_2_20220625_173102,0.0661231,0.825966,0.0783862,0.331149,0.131468,0.0172839,3702,0.005715,DeepLearning
XGBoost_1_AutoML_2_20220625_173102,0.0658629,0.795778,0.116098,0.417953,0.168336,0.0283371,23291,0.008447,XGBoost
XGBoost_3_AutoML_2_20220625_173102,0.0635095,0.781511,0.126929,0.432541,0.171993,0.0295815,24195,0.015282,XGBoost


In [41]:
m = aml.get_best_model(criterion = "mean_per_class_error")
m = h2o.get_model("DeepLearning_1_AutoML_2_20220625_173102")

In [42]:
# exm = m.explain(test)

In [43]:
preds = m.predict(test)

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


In [44]:
preds 

predict,p0,p1
1,0.977796,0.022204
0,1,8.43577e-09
0,0.995093,0.00490747
1,0.903512,0.0964881
0,0.999986,1.35057e-05
0,0.999965,3.53816e-05
0,0.999999,9.015e-07
0,1,4.72272e-14
0,1,2.23677e-16
0,0.999764,0.00023588


In [45]:
import sklearn.metrics
pred2 =  h2o.as_list(preds,use_pandas=True)
test2 =h2o.as_list(test,use_pandas=True)
y_true = test2['stroke'].to_numpy()
y_pred = pred2['predict'].to_numpy()
mat = sklearn.metrics.confusion_matrix(y_true, y_pred)

In [46]:
mat

array([[6128, 2587],
       [  30,  131]])

In [47]:
sklearn.metrics.balanced_accuracy_score(y_true, y_pred)

0.7584100376661926

In [48]:
(y_true==y_pred).mean()

0.7051599819738621

In [49]:
h2o.save_model(m,path = "s3://science-hackathon-2022-duc-nguyen/model_bin")

's3://science-hackathon-2022-duc-nguyen/model_bin/DeepLearning_1_AutoML_2_20220625_173102'